### Goals

1. Write a program that can create a list of all nxn latin squares, call it Ln. This is probably only computationally feasible up to around 6x6.
2. Write a function that "standardizes" a latin square by permuting the symbols such that the first row of the square is in order, 1, 2, 3, ..., n. Call this function F()
3. Work through the following algorithm, starting at 4x4 latin squares. 5x5 should also be possible, 6x6 maybe not. 
    - Select a latin square from Ln, call it Sj.
    - Make a list of all latin squares that can be obtained from Sj by permuting the rows and columns, call this Aj.
    - Count how many squares there are and how many of them are equal to Sj.
    - Remove the squares in Aj from Ln and repeat a-c until there are none left. Count how many of these different "classes" there are and how big each class is.

All of this could also be adjusted to standardize all the squares by F().

In [1]:
from itertools import permutations
import random

### Misc Functions

In [19]:
def print_square(square):
    '''
    Print squre row by row for better readability.
    '''
    for row in square:
        print(row)
    print()

### 1. Generating Latin Squares

In [20]:
def Ln(n: int) -> list:
    '''
    Generates all Latin squares of order n.
    '''
    all_rows = list(permutations(range(1,n+1)))
    squares = []

    def backtrack(square, depth):
        if depth == n:
            squares.append([list(row) for row in square])
            return
        
        for row in all_rows:
            if all(row[col] not in used_in_col[col] for col in range(n)):
                square.append(row)
                for col in range(n):
                    used_in_col[col].add(row[col])

                backtrack(square, depth + 1)

                for col in range(n):
                    used_in_col[col].remove(row[col])
                square.pop()
    used_in_col = [set() for _ in range(n)]

    backtrack([],0)
    return squares

def Ln_reduced(n: int,) -> list:
    '''
    Generates all reduced Latin squares of order n,
    fixing the first row to [1, 2, ..., n].
    This removes symmetries due to row permutations.
    '''
    all_rows = list(permutations(range(1, n + 1)))
    squares = []

    def backtrack(square, depth):
        if depth == n:
            squares.append(square[:])
            return

        for row in all_rows:
            if all(row[col] not in used_in_col[col] for col in range(n)):
                square.append(row)
                for col in range(n):
                    used_in_col[col].add(row[col])

                backtrack(square, depth + 1)

                for col in range(n):
                    used_in_col[col].remove(row[col])
                square.pop()

    first_row = tuple(range(1, n + 1))
    used_in_col = [set() for _ in range(n)]
    for col in range(n):
        used_in_col[col].add(first_row[col])  # track first row values
        
    backtrack([first_row], 1)
    return squares
    

#### 1. Test Cases

In [5]:
three_by_three_reduced = Ln_reduced(3) 
three_by_three = Ln(3)
print(f"Total latin square of order 3: {len(three_by_three)}")
print(f"Reduced latin square of order 3: {len(three_by_three_reduced)}")

for square in three_by_three:
    #print_square(square)
    continue

Total latin square of order 3: 12
Reduced latin square of order 3: 2


In [21]:
four_by_four = Ln(4)
four_by_four_reduced = Ln_reduced(4)
print(f"Total latin square of order 4: {len(four_by_four)}")
print(f"Reduced latin square of order 4: {len(four_by_four_reduced)}")
for square in four_by_four:
    #print_square(square)
    continue
# 576 squares total
# 24 reduced

Total latin square of order 4: 576
Reduced latin square of order 4: 24


In [7]:
five_by_five = Ln(5)
five_by_five_reduced = Ln_reduced(5)
print(f"Total latin square of order 5: {len(five_by_five)}")
print(f"Reduced latin square of order 5: {len(five_by_five_reduced)}")
for square in five_by_five:
    #print_square(square)
    continue
# 161280 Squares
# 1344 total

Total latin square of order 5: 161280
Reduced latin square of order 5: 1344


In [ ]:
six_by_six = Ln(6)
six_by_six_reduced = Ln_reduced(6)
print(f"Total latin square of order 6: {len(six_by_six)}")
print(f"Total latin square of order 6: {len(six_by_six_reduced)}")
for square in six_by_six:
    #print_square(square)
    continue

### 2. "Standardize" Squares (Symmetry Reduction)

In [8]:
def Fn(square: list) -> list:
    '''
    Standardize a Latin square so that the first row is [1, 2, ..., n].
    '''
    first_row = square[0]

    # Creates a mapping dict for standardization of square
    row_mapping = {val: i + 1 for i, val in enumerate(first_row)}

    # Apply the mapping to the entire square
    standardized_square = [[row_mapping[val] for val in row] for row in square]

    return standardized_square

##### 2. Test Cases

In [ ]:
square1 = [
    [3, 1, 2],
    [2, 3, 1],
    [1, 2, 3]
]

''' 
Expected output:
    [1, 2, 3],
    [3, 1, 2], 
    [2, 3, 1] ] 
'''

square2 = [
    [4, 1, 3, 2],
    [2, 4, 1, 3],
    [1, 3, 2, 4],
    [3, 2, 4, 1]
]

''' 
Expected output:
    [1, 2, 3, 4],
    [4, 1, 2, 3],
    [2, 3, 4, 1],
    [3, 4, 1, 2] 
'''

square3 = [
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 1],
    [3, 4, 5, 1, 2],
    [4, 5, 1, 2, 3],
    [5, 1, 2, 3, 4]
]
'''
Already "standardized," expected output = input
'''

for i in [square1, square2, square3]:
    result = Fn(i)
    print_square(result)

[1, 2, 3]
[3, 1, 2]
[2, 3, 1]

[1, 2, 3, 4]
[4, 1, 2, 3]
[2, 3, 4, 1]
[3, 4, 1, 2]

[1, 2, 3, 4, 5]
[2, 3, 4, 5, 1]
[3, 4, 5, 1, 2]
[4, 5, 1, 2, 3]
[5, 1, 2, 3, 4]



### 3. Main Algorithm

#### Permutation Functions

In [9]:
def row_col_permutations(square: list) -> list:
    '''
    Generate all Latin squares by permuting rows and columns of the given square.
    Returns a list of all Latin squares generated for row/column permutations of passed square.
    '''
    n = len(square)
    perms = set()

    for row_perm in permutations(range(n)):
        # Apply row permutation
        permuted_rows = [square[i] for i in row_perm]

        for col_perm in permutations(range(n)):
            # Apply column permutation to each row
            permuted_square = [
                [row[j] for j in col_perm]
                for row in permuted_rows
            ]
            # Convert to tuple of tuples for hashing
            perms.add(tuple(tuple(row) for row in permuted_square))

    # Return list of unique permutations
    return [ [list(row) for row in square] for square in perms ]


In [10]:
def symbol_permutations(square:list , mapping: dict) -> list:
    '''
    Applies a symbol permutation to a Latin square.

    parameters:
        symbol_perm (dict): A mapping from original symbol to new symbol.
    '''
    return [[mapping[val] for val in row] for row in square]

In [11]:
def all_symbol_permutations(square:list) -> list:
    '''
    Generates all unique Latin squares via symbol permutations.
    '''
    n = len(square)
    perms = permutations(range(1, n + 1))
    seen = set()
    unique_squares = []

    for p in perms:
        mapping = {old: new for old, new in zip(range(1, n + 1), p)}
        permuted = symbol_permutations(square, mapping)
        flat = tuple(tuple(row) for row in permuted)
        if flat not in seen:
            seen.add(flat)
            unique_squares.append(permuted)

    return unique_squares

In [ ]:
def isotopy(n:int):
    '''
    Finds isotopy classes (row/col/symbol permutations).
    '''
    # Get all Latin squares
    squares = Ln(n)
    # covert to tuples to store in set 
    remaining = set(tuple(map(tuple, square)) for square in squares)  # hashable form (immutable)
    classes = []

    row_perms = list(permutations(range(n))) # 0 - 3
    col_perms = list(permutations(range(n))) # 0 - 3
    sym_perms = list(permutations(range(1, n + 1))) # 1 - 4


    while remaining:
        # Pick an arbitrary square from remaining
        Sj = random.choice(list(remaining))
        Sj_matrix = [list(row) for row in Sj]

        isotopy_class = set()

        # Apply all triples (row_perm, col_perm, sym_perm) as the isotopy group action
        for r_perm in row_perms:
            for c_perm in col_perms:
                for s_perm in sym_perms:
                    # Create symbol mapping (dict) from original symbol to permuted symbol
                    sym_map = {orig: new for orig, new in zip(range(1, n + 1), s_perm)}

                    # Apply row permutation
                    row_permuted = [Sj_matrix[r_perm[i]] for i in range(n)]

                    # Apply column permutation
                    col_permuted = [[row[c_perm[j]] for j in range(n)] for row in row_permuted]

                    # Apply symbol permutation
                    final = [[sym_map[val] for val in row] for row in col_permuted]

                    isotopy_class.add(tuple(tuple(row) for row in final))

        # Record the class
        classes.append([list(map(list, square)) for square in isotopy_class])

        # Remove entire isotopy class from remaining
        remaining -= isotopy_class

    class_sizes = [len(c) for c in classes]
    return classes, class_sizes

In [ ]:
def equivalence(n:int):
    '''
    Finds row/col equivalence classes.
    '''
    # Step 1: Generate latin squares of order n
    squares = Ln(n)
    # covert to tuples to store in set 
    remaining = set(tuple(map(tuple, square)) for square in squares)  # hashable form
    classes = []

    while remaining:
        # Step 2: Pick an arbitrary square Sj from the remaining ones
        Sj = random.choice(list(remaining))
        Sj_matrix = [list(row) for row in Sj]  # convert back to list of lists
        
        # Step 3. Generate its equivalence class by row/column permutations
        Aj = row_col_permutations(Sj_matrix)
        Aj_set = set(tuple(map(tuple, square)) for square in Aj)
        
        # Step 4. Record the class
        classes.append([list(map(list, square)) for square in Aj_set])
        remaining -= Aj_set # Remove this entire class from the remaining pool


    # Optionally, return sizes
    class_sizes = [len(c) for c in classes]
    return classes, class_sizes

#### 3. Test Cases

In [78]:
eq_classes_3x3, eq_class_sizes_3x3 = equivalence(3)
print("Number of equivalence classes:", len(eq_classes_3x3))
print("Sizes of classes:", eq_class_sizes_3x3)
print()
isotopy_classes_3x3, isotopy_class_sizes_3x3 = isotopy(3)
print("Number of isotopy classes:", len(isotopy_classes_3x3))
print("Sizes of classes:", isotopy_class_sizes_3x3)

print("\nTotal squares covered:", sum(eq_class_sizes_3x3), "&", sum(isotopy_class_sizes_3x3))



Number of equivalence classes: 1
Sizes of classes: [12]

Number of isotopy classes: 1
Sizes of classes: [12]

Total squares covered: 12 & 12


In [79]:
eq_classes_4x4, eq_class_sizes_4x4 = equivalence(4)
print("Number of equivalence classes:", len(eq_classes_4x4))
print("Sizes of classes:", eq_class_sizes_4x4)
print()
isotopy_classes_4x4, isotopy_class_sizes_4x4 = isotopy(4)
print("Number of isotopy classes:", len(isotopy_classes_4x4))
print("Sizes of classes:", isotopy_class_sizes_4x4)

print("\nTotal squares covered:", sum(eq_class_sizes_4x4), "&", sum(isotopy_class_sizes_4x4))

Number of equivalence classes: 4
Sizes of classes: [144, 144, 144, 144]

Number of isotopy classes: 2
Sizes of classes: [144, 432]

Total squares covered: 576 & 576


In [80]:
eq_classes_5x5, eq_class_sizes_5x5 = equivalence(5)
print("Number of equivalence classes:", len(eq_classes_5x5))
print("Sizes of classes:", eq_class_sizes_5x5)
print()
isotopy_classes_5x5, isotopy_class_sizes_5x5 = isotopy(5)
print("Number of isotopy classes:", len(isotopy_classes_5x5))
print("Sizes of classes:", isotopy_class_sizes_5x5)

print("\nTotal squares covered:", sum(eq_class_sizes_5x5), "&", sum(isotopy_class_sizes_5x5))

Number of equivalence classes: 16
Sizes of classes: [14400, 14400, 2880, 14400, 14400, 2880, 14400, 14400, 14400, 2880, 14400, 2880, 2880, 14400, 14400, 2880]

Number of isotopy classes: 2
Sizes of classes: [144000, 17280]

Total squares covered: 161280 & 161280
